# [Issue 98](https://github.com/mjuenema/python-terrascript/issues/98): Change to .update() in 0.8.0? 

## Problem

> In our Terrascript 0.6.1 scripts, we have a bunch of functions that define pieces of the configuration, and we 
> assemble them like this:

```python
ts = terrascript.Terrascript()
ts += terrascript.terraform(
  backend = terrascript.backend(
    "s3",
    **config['backend']['s3']
    )
  )
ts.update(tsumami.provider.define(config))
ts.update(tsumami.s3.define(config))
ts.update(tsumami.security_groups.define(config))
```

> where `config` is a dict based on a collection of YAML config files.
>
> Each of those functions looks like this:

```python
def define(config):
  my_ts = terrascript.Terrascript()
  if 's3-bucket' in config:
    for (resource, data) in config['s3-bucket'].items():
      # code that does various things
      my_ts += terrascript.resource.aws.aws_s3_bucket(
        resource,
        bucket = bucketname,
        **data
        )
  return(my_ts)
```

> That used to work in 0.6.1, but doesn't in 0.8.0; it seems that each `ts.update(something)` replaces the
> previous one, and only the last one ends up in the config when we write out `str(ts)` at the end.
>
> What's the right way to do this in 0.8.0?

## Analysis

The problem is caused by having removed the ``Terrascript.update()`` method in release 0.8.0.

```console
$ git diff 0.6.1 0.8.0 terrascript/__init__.py
-    def update(self, terrascript2):
-        if isinstance(terrascript2, Terrascript):
-            for item in terrascript2._item_list:
-                self.__add__(item)
-        else:
-            raise TypeError('{0} is not a Terrascript instance.'.format(
-                type(terrascript2)))
```

In fact, there is still a ``Terrascript.update()`` method but it is now inherited from the Python dictionary class which works of course completely differently.


**Terrascript 0.8.0**
```python
class Terrascript(dict):
    ...
```

**Terrascript 0.6.1**
```python
class Terrascript(object)
```

The obvious fix is to restore the ``Terrascript.update()`` method. This is not a simply copy and paste 
job as the ``Terrascript._item_list`` attribute has also been removed.

In Terrascript 0.6.1 ``Terrascript.__item_list`` was managed behind the scenes. Terrascript 0.8.0 removed all this logic with the objective of making it much simpler.

**Terrascript 0.6.1**

```python
class Terrascript(object):
    """Top-level container for Terraform configurations."""

    def __init__(self):
        ...
        self._item_list = []
        
    def __add__(self, item):
        ...
        if not isinstance(item, Terrascript):
            if item in self._item_list:
                self._item_list.remove(item)
            self._item_list.append(item)
```

Since release 0.8.0 the ``Terrascript`` class is just a Python dictionary with extra smarts the best approach would be to make ``Terrascript.update()`` perform a "nested dictionary merge". That should hopefully provide the
desired result.

### Current behaviour

In [108]:
import terrascript
import terrascript.provider
import terrascript.resource
import terrascript.data

First ``Terrascript`` instance.

In [109]:
config1 = terrascript.Terrascript()
config1 += terrascript.provider.aws(alias="east", region="us-east-1")
config1

{'provider': {'aws': [{'alias': 'east', 'region': 'us-east-1'}]}}

Second ``Terrascript`` instance.

In [110]:
config2 = terrascript.Terrascript()
config2 += terrascript.provider.aws(alias="east", region="us-west-1")
config2

{'provider': {'aws': [{'alias': 'east', 'region': 'us-west-1'}]}}

Calling the (dictionary) ``.update()`` method overwrites the ``config['provider']`` key of the first instance with the value of the second instance.

In [111]:
config1.update(config2)
config1

{'provider': {'aws': [{'alias': 'east', 'region': 'us-west-1'}]}}

### Draft solution
It may be possible to implement ``.update()`` by "recursing" into ``config2`` to find all objects that are 
instances of sub-classes of ``Block`` and them ``_add__()`` them to ``self``, i.e. ``config1``. Unfortunately ``Provisioner`` is currently not derived from ``Block``.

In [112]:
config1 = terrascript.Terrascript()
config1 += terrascript.provider.aws(alias="east", region="us-east-1")
config2 = terrascript.Terrascript()
config2 += terrascript.provider.aws(alias="east", region="us-west-1")

In [113]:
def recurse(o):
    if isinstance(o, terrascript.Block):
        yield o
    elif isinstance(o, dict):
        for k,v in o.items():
            yield from recurse(v)
    elif isinstance(o, list):
        for i in o:
            yield from recurse(i)
    
    return

for x in recurse(config2):
    config1 += x
    
print(config1)

{
  "provider": {
    "aws": [
      {
        "alias": "east",
        "region": "us-east-1"
      },
      {
        "alias": "east",
        "region": "us-west-1"
      }
    ]
  }
}


That works as expected. This is a candidate for the new ``update()`` method.
In addition ``Provisioner`` will have to be derived from ``Block`` and not from ``dict``.

## Solution
* Implement ``update()`` method.
* Implement ``__iter__()`` method which is used by ``update()``

## ``__iter__()`` method
``__iter__()`` iterates over all "top-level" blocks: ``Resource``, ``Data``, ``Provider``, ``Variable``, ``Module`` and ``Output``.

In [2]:
import terrascript
import terrascript.provider
import terrascript.resource
import terrascript.data

config = terrascript.Terrascript()

# Google Cloud Compute provider
config += terrascript.provider.google(
    credentials='${file("account.json")}', project="myproject", region="us-central1"
)

# Google Compute Image (Debian 9) data source
config += terrascript.data.google_compute_image("image", family="debian-9")

# Add Google Compute Instance resource
config += terrascript.resource.google_compute_instance(
    "myinstance",
    name="myinstance",
    machine_type="n1-standard-1",
    zone="us-central1-a",
    boot_disk={
        "initialize_params": {"image": "data.google_compute_image.image.self_link"}
    },
    network_interface={"network": "default", "access_config": {}},
)

for o in config:
    print(type(o))

<class 'terrascript.provider.google.google'>
<class 'terrascript.data.google.google_compute_image'>
<class 'terrascript.resource.google.google_compute_instance'>


## ``update()`` method

In [3]:
config1 = terrascript.Terrascript()
config1 += terrascript.provider.aws(alias="east", region="us-east-1")

config2 = terrascript.Terrascript()
config2 += terrascript.provider.aws(alias="east", region="us-west-1")

config1.update(config2)
config1

{'provider': {'aws': [{'alias': 'east', 'region': 'us-east-1'},
   {'alias': 'east', 'region': 'us-west-1'}]}}